In [1]:
from pathlib import Path
import sys

sys.path.append('/eos/home-i03/m/morwat/.local/lib/python3.9/site-packages/')

In [2]:
home_path = str(Path.cwd().parent)
sys.path.append(home_path)

In [3]:
from aper_package.aperture_data import Data
from aper_package.plotting import plot
from aper_package.timber_data import BPMData
from aper_package.timber_data import CollimatorsData

In [4]:
aper = Data(path_b1=home_path+'/test_data/Martas_injection_b1.json')
aper.load_aperture(path_b1=home_path+'/test_data/all_optics_B1.tfs')

Loading line from dict:   0%|          | 0/102050 [00:00<?, ?it/s]

Done loading line from dict.           


Loading line from dict:   0%|          | 0/102122 [00:00<?, ?it/s]

Done loading line from dict.           
Computing twiss for beam 1...
Compiling ContextCpu kernels...
Done compiling ContextCpu kernels.
Computing twiss for beam 2...
Compiling ContextCpu kernels...
Done compiling ContextCpu kernels.
Done computing twiss.


In [7]:
data = BPMData(spark)
col_data = CollimatorsData(spark)

In [ ]:
plot(aper, 'h', BPM_data=None, collimator_data=None)